In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os
#from transformers import pipeline
#import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#pipe = pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device="cuda",)

def generate_prompts(image_path):
    # Function to generate 20 prompts using LLaMA 3
    try:
        messages = [{"role": "system", "content": "You are Generate 20 relevant prompts with no more than 6 words per prompt for an image intended for zero shot detection model's text encoder"},
                    {"role": "user", "content": f"The goal is to detect {prompt}, here is the image: {image_path}"},
                    ]
        #terminators = [pipe.tokenizer.eos_token_id, pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        #outputs = pipe(messages, max_new_tokens=500, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
        #assistant_response = outputs[0]["generated_text"][-1]["content"]
        #prompts = assistant_response.split('\n')

        #return [p.strip() for p in prompts if p.strip()]

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")
        return []

def show_window(window):
    window.attributes('-fullscreen', True)
    window.update_idletasks()  # Ensure the window is fully rendered

def go_back(window):
    root.deiconify()
    root.attributes('-fullscreen', True)
    window.destroy()

def select_manual():
    root.withdraw()
    manual_window = tk.Toplevel(root)
    manual_window.title("Manual Prompt and Confidence Tuning")
    manual_window.configure(bg="black")
    show_window(manual_window)

    def select_folder():
        folder = filedialog.askdirectory(parent=manual_window)
        if folder:
            images = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            if images:
                display_image(images[0])
                current_image_index.set(0)
                image_list.set(images)
            else:
                messagebox.showwarning("No Images", "The selected folder does not contain any images.")

    def display_image(image_path):
        img = tk.PhotoImage(file=image_path)
        image_label.configure(image=img)
        image_label.image = img

    def next_image():
        index = current_image_index.get() + 1
        images = image_list.get()
        if index < len(images):
            display_image(images[index])
            current_image_index.set(index)
        else:
            messagebox.showinfo("End of Images", "No more images in the folder.")

    def auto_annotate_remaining():
        output_folder = filedialog.askdirectory(parent=manual_window)
        if output_folder:
            # Add your auto-annotation logic
            messagebox.showinfo("Auto Annotation", "Annotations saved to the output folder.")

    back_btn = tk.Button(manual_window, text="Back", command=lambda: go_back(manual_window), bg="grey", fg="white", width=10, height=2, font=("Helvetica", 16))
    back_btn.place(x=10, y=10)

    folder_btn = tk.Button(manual_window, text="Select Image Folder", command=select_folder, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
    folder_btn.pack(pady=20)

    current_image_index = tk.IntVar(value=0)
    image_list = tk.StringVar(value=[])

    image_label = tk.Label(manual_window, bg="black")
    image_label.pack(side=tk.RIGHT, padx=10)

    prompt_label = tk.Label(manual_window, text="Enter Prompt:", bg="black", fg="white", font=("Helvetica", 16))
    prompt_label.pack(anchor=tk.W, padx=10)

    prompt_entry = tk.Entry(manual_window, width=50, font=("Helvetica", 16))
    prompt_entry.pack(anchor=tk.W, padx=10, pady=5)

    confidence_label = tk.Label(manual_window, text="Confidence:", bg="black", fg="white", font=("Helvetica", 16))
    confidence_label.pack(anchor=tk.W, padx=10)

    confidence_slider = tk.Scale(manual_window, from_=0, to=100, orient=tk.HORIZONTAL, length=400, font=("Helvetica", 16))
    confidence_slider.pack(anchor=tk.W, padx=10, pady=5)

    # Create a frame for bottom buttons to be placed in the corners
    bottom_frame = tk.Frame(manual_window, bg="black")
    bottom_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=20)

    next_btn = tk.Button(bottom_frame, text="Next Image", command=next_image, bg="green", fg="white", width=20, height=3, font=("Helvetica", 16))
    next_btn.pack(side=tk.LEFT, padx=10, pady=10, anchor="sw")

    auto_annotate_btn = tk.Button(bottom_frame, text="Auto Annotate Remaining", command=auto_annotate_remaining, bg="red", fg="white", width=20, height=3, font=("Helvetica", 16))
    auto_annotate_btn.pack(side=tk.RIGHT, padx=10, pady=10, anchor="se")

def select_automated():
    root.withdraw()
    automated_window = tk.Toplevel(root)
    automated_window.title("Automated Prompt and Confidence Tuning")
    automated_window.configure(bg="black")
    show_window(automated_window)

    def select_labelled_images():
        response = messagebox.askyesno("Labelled Images", "Do you have labelled images?", parent=automated_window)
        if response:
            labelled_folder = filedialog.askdirectory(title="Select Labelled Image Folder", parent=automated_window)
            if labelled_folder:
                labelled_folder_label.config(text=f"Labelled Folder: {labelled_folder}")

    def select_unannotated_folder():
        unannotated_folder = filedialog.askdirectory(title="Select Unannotated Image Folder", parent=automated_window)
        if unannotated_folder:
            unannotated_folder_label.config(text=f"Unannotated Folder: {unannotated_folder}")

    def select_output_folder():
        output_folder = filedialog.askdirectory(title="Select Output Folder", parent=automated_window)
        if output_folder:
            output_folder_label.config(text=f"Output Folder: {output_folder}")

    def perform_automatic_annotation():
        messagebox.showinfo("Automated Annotation", "Annotations saved to the output folder.", parent=automated_window)

    def prompt_selection():
        if not hasattr(automated_window, "prompt_buttons_added"):
            list_prompts_btn = tk.Button(right_frame, text="List of Prompts", command=handle_list_of_prompts, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
            list_prompts_btn.pack(pady=10)

            generate_prompts_btn = tk.Button(right_frame, text="Generate Prompts", command=handle_generate_prompts, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
            generate_prompts_btn.pack(pady=10)

            automated_window.prompt_buttons_added = True

    def handle_list_of_prompts():
        messagebox.showinfo("Prompt List", "Selected list of prompts.", parent=automated_window)

    def handle_generate_prompts():
        sample_image_path = filedialog.askopenfilename(title="Select Sample Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")], parent=automated_window)
        if sample_image_path:
            prompt = "a sample prompt"  # Update this as needed
            prompts = generate_prompts(sample_image_path, prompt)
            if prompts:
                messagebox.showinfo("Generated Prompts", "\n".join(prompts), parent=automated_window)

    left_frame = tk.Frame(automated_window, bg="black")
    left_frame.pack(side=tk.LEFT, padx=10, pady=10, expand=True)

    right_frame = tk.Frame(automated_window, bg="black")
    right_frame.pack(side=tk.RIGHT, padx=10, pady=10, expand=True)

    back_btn = tk.Button(automated_window, text="Back", command=lambda: go_back(automated_window), bg="grey", fg="white", width=10, height=2, font=("Helvetica", 16))
    back_btn.place(x=10, y=10)

    label_btn = tk.Button(left_frame, text="Select Labelled Images", command=select_labelled_images, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
    label_btn.pack(pady=20)

    labelled_folder_label = tk.Label(left_frame, text="", bg="black", fg="white", font=("Helvetica", 16))
    labelled_folder_label.pack(pady=5)

    unannotated_btn = tk.Button(left_frame, text="Select Unannotated Image Folder", command=select_unannotated_folder, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
    unannotated_btn.pack(pady=20)

    unannotated_folder_label = tk.Label(left_frame, text="", bg="black", fg="white", font=("Helvetica", 16))
    unannotated_folder_label.pack(pady=5)

    output_btn = tk.Button(left_frame, text="Select Output Folder", command=select_output_folder, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
    output_btn.pack(pady=20)

    output_folder_label = tk.Label(left_frame, text="", bg="black", fg="white", font=("Helvetica", 16))
    output_folder_label.pack(pady=5)

    prompt_select_btn = tk.Button(right_frame, text="Prompt Selection", command=prompt_selection, bg="blue", fg="white", width=30, height=3, font=("Helvetica", 16))
    prompt_select_btn.pack(pady=20)

    start_btn = tk.Button(automated_window, text="Start Annotation", command=perform_automatic_annotation, bg="green", fg="white", width=30, height=3, font=("Helvetica", 16))
    start_btn.pack(side=tk.BOTTOM, pady=20)

# Splash screen
def show_splash_screen():
    splash = tk.Toplevel()
    splash.title("Splash Screen")
    splash.attributes('-fullscreen', True)
    splash.configure(bg="black")

    # Load and display the team logo
    logo_path = "AMS_Logo_Final_Removed.png"  # Replace with the path to your logo file
    logo_image = Image.open(logo_path)

    # Scale the image to fit the screen while maintaining the aspect ratio
    screen_width = splash.winfo_screenwidth()
    screen_height = splash.winfo_screenheight()
    logo_width, logo_height = logo_image.size
    aspect_ratio = logo_width / logo_height

    if screen_width / aspect_ratio <= screen_height:
        new_width = screen_width
        new_height = int(screen_width / aspect_ratio)
    else:
        new_height = screen_height
        new_width = int(screen_height * aspect_ratio)

    logo_image = logo_image.resize((new_width, new_height), Image.LANCZOS)

    logo = ImageTk.PhotoImage(logo_image)
    logo_label = tk.Label(splash, image=logo, bg="black")
    logo_label.image = logo  # Keep a reference to avoid garbage collection
    logo_label.pack(expand=True)

    # Automatically close the splash screen after 4 seconds
    splash.after(4000, lambda: (splash.destroy(), show_main_window()))

def show_main_window():
    root.deiconify()
    root.attributes('-fullscreen', True)

# Main window
root = tk.Tk()
root.title("Prompt and Confidence Tuning")
root.attributes('-fullscreen', True)
root.configure(bg="black")
root.withdraw()

# Display splash screen
show_splash_screen()

# Centering the buttons
button_frame = tk.Frame(root, bg="black")
button_frame.pack(expand=True)

exit_btn = tk.Button(root, text="Exit", command=root.destroy, bg="red", fg="white", width=10, height=2, font=("Helvetica", 16))
exit_btn.place(x=10, y=10)

manual_btn = tk.Button(button_frame, text="Manual Prompt and Confidence Tuning", command=select_manual, bg="blue", fg="white", width=40, height=3, font=("Helvetica", 20))
manual_btn.pack(pady=20)

automated_btn = tk.Button(button_frame, text="Automated Prompt and Confidence Tuning", command=select_automated, bg="red", fg="white", width=40, height=3, font=("Helvetica", 20))
automated_btn.pack(pady=20)

root.mainloop()